<a href="https://colab.research.google.com/github/Yundo37/automation-portfolio-Yundo/blob/main/Test_Automation_Yundo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Selenium 테스트 자동화 포트폴리오 - Live Demo
# Author: 윤도현 (Dohyeon Yun) | QA Engineer
# Email: porore37@naver.com | Phone: +82 10 5652 3436

print("=" * 60)
print("Selenium 테스트 자동화 포트폴리오")
print("QA Engineer: 윤도현 (porore37@naver.com)")
print("GitHub: https://github.com/Yundo37/automation-portfolio-Yundo")
print("=" * 60)

import subprocess
import sys
import os
from datetime import datetime

# 1. Chrome 설치
print("\n1단계: Chrome 브라우저 설치")
subprocess.run(['apt-get', 'update', '-qq'], capture_output=True)
subprocess.run(['apt-get', 'install', '-y', '-qq', 'chromium-browser'], capture_output=True)
print("Chrome 설치 완료")

# 2. GitHub 레포지토리 클론
print("\n2단계: GitHub 레포지토리 클론")
repo_url = "https://github.com/Yundo37/automation-portfolio-Yundo.git"
print(f"클론 시도: {repo_url}")

if os.path.exists("automation-portfolio-Yundo"):
    subprocess.run(['rm', '-rf', 'automation-portfolio-Yundo'], check=True)

result = subprocess.run(['git', 'clone', repo_url], capture_output=True, text=True)

if result.returncode == 0:
    print("GitHub 레포지토리 클론 성공")
    os.chdir("automation-portfolio-Yundo")
    print(f"작업 디렉토리: {os.getcwd()}")
else:
    print("GitHub 클론 실패 - 데모를 위해 로컬 파일 생성으로 전환")
    print(f"오류: {result.stderr}")

# 3. 프로젝트 구조 확인
print("\n3단계: 프로젝트 구조 확인")
def show_structure():
    for root, dirs, files in os.walk('.'):
        level = root.replace('.', '').count(os.sep)
        indent = '  ' * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = '  ' * (level + 1)
        for file in files:
            if not file.startswith('.') and not file.endswith('.pyc'):
                print(f"{subindent}{file}")

show_structure()

# 4. 패키지 설치
print("\n4단계: 필요한 패키지 설치")
if os.path.exists("requirements.txt"):
    print("requirements.txt에서 패키지 설치...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt", "-q"])
    print("패키지 설치 완료")
else:
    print("기본 패키지 설치...")
    packages = ["selenium==4.15.0", "pytest==7.4.3", "pytest-html==4.1.1", "webdriver-manager==4.0.1"]
    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
    print("기본 패키지 설치 완료")

# 5. 테스트 실행
print("\n5단계: 테스트 실행")
start_time = datetime.now()
print(f"시작 시간: {start_time.strftime('%H:%M:%S')}")

if os.path.exists("tests"):
    print("전체 테스트 스위트 실행...")

    result = subprocess.run([
        sys.executable, "-m", "pytest",
        "tests/",
        "--html=reports/full_report.html",
        "--self-contained-html",
        "-v", "-s"
    ], capture_output=True, text=True)

    print("테스트 실행 결과:")
    print(result.stdout)

    if result.stderr:
        print("오류 메시지:")
        print(result.stderr)
else:
    print("테스트 파일을 찾을 수 없습니다")

end_time = datetime.now()
print(f"완료 시간: {end_time.strftime('%H:%M:%S')}")
print(f"총 실행 시간: {end_time - start_time}")

# 6. 결과 확인
print("\n6단계: 결과 확인")
reports_dir = "reports"
if os.path.exists(reports_dir):
    print(f"{reports_dir} 디렉토리 내용:")
    for file in os.listdir(reports_dir):
        file_path = os.path.join(reports_dir, file)
        if os.path.isfile(file_path):
            size = os.path.getsize(file_path)
            print(f"  {file} ({size:,} bytes)")

screenshots = [f for f in os.listdir('.') if f.endswith('.png')]
if screenshots:
    print(f"\n캡처된 스크린샷: {len(screenshots)}개")
    for screenshot in screenshots:
        print(f"  {screenshot}")

print("\n" + "=" * 60)
print("테스트 자동화 포트폴리오 데모 완료")
print("=" * 60)
print("시연된 주요 기능:")
print("  - Page Object Model 패턴")
print("  - HTML 리포트 생성")
print("  - 실패 시 자동 스크린샷 캡처")
print("  - 파라미터화된 테스트 데이터")
print("  - 의도적 실패 케이스")
print("\n연락처:")
print("  Email: porore37@naver.com")
print("  Phone: +82 10 5652 3436")
print("  GitHub: https://github.com/Yundo37/automation-portfolio-Yundo")
print("\n포트폴리오를 확인해주셔서 감사합니다")

Selenium 테스트 자동화 포트폴리오
QA Engineer: 윤도현 (porore37@naver.com)
GitHub: https://github.com/Yundo37/automation-portfolio-Yundo

1단계: Chrome 브라우저 설치
Chrome 설치 완료

2단계: GitHub 레포지토리 클론
클론 시도: https://github.com/Yundo37/automation-portfolio-Yundo.git
GitHub 레포지토리 클론 성공
작업 디렉토리: /content/automation-portfolio-Yundo

3단계: 프로젝트 구조 확인
./
  requirements.txt
  conftest.py
  README.md
  pageObjects/
    HomePage.py
    CheckoutPage.py
    ConfirmPage.py
  .git/
    packed-refs
    description
    index
    HEAD
    config
    logs/
      HEAD
      refs/
        heads/
          main
        remotes/
          origin/
            HEAD
    refs/
      heads/
        main
      remotes/
        origin/
          HEAD
      tags/
    info/
      exclude
    hooks/
      pre-applypatch.sample
      pre-rebase.sample
      commit-msg.sample
      prepare-commit-msg.sample
      fsmonitor-watchman.sample
      pre-merge-commit.sample
      post-update.sample
      pre-push.sample
      applypatch-msg.sampl